In [ ]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================


    # ## load chosen model
    # # =========================
    # if chosen_model == 'resnet50':
    #     preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # # elif chosen_model == 'resnet152':
    # #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    # #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    # elif chosen_model == 'vgg16':
    #     preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    # elif chosen_model == 'vgg19':
    #     preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    # elif chosen_model == 'mobilenet':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    # elif chosen_model == 'mobilenet_v2':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    # elif chosen_model == 'efficientnet':
    #     preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    # elif chosen_model == 'inception_v3':
    #     preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    # else:
    #     raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    # test_acc = []
    # test_f1 = []
    front_test_acc = []
    side_test_acc = []
    soft_test_acc = []
    hard_test_acc = []

    front_test_f1 = []
    side_test_f1 = []
    soft_test_f1 = []
    hard_test_f1 = []
    
    vote_type_arr_acc = [front_test_acc, side_test_acc, soft_test_acc, hard_test_acc]
    vote_type_arr_f1 = [front_test_f1, side_test_f1, soft_test_f1, hard_test_f1]
    # vote_type_arr = ["front", "side", "soft", "hard", "product"]
    vote_type_arr = ["front", "side", "soft", "hard"]


    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]


        ## train, validation image
        # =========================
        ## train front and side
        save_path_arr = ["front", "side"]
        for save_path in save_path_arr:
            ## 分資料
            # =========================
            if save_path == "side":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("front", "side")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("front", "side")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("front", "side")
                preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
                pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0

            if save_path == "front":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("side", "front")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("side", "front")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("side", "front")
                preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
                pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50

            print(len(k_fold_train))
            print(len(k_fold_val))


            # 確認
            print(k_fold_train.iloc[0]['Filepath'].split("\\")[-3])
            print(test_df.iloc[0]['Filepath'].split("\\")[-3])
            
            ## 讀資料
            # =========================
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
            train_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_train,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            val_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_val,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
            test_images = test_generator.flow_from_dataframe(
                dataframe=test_df,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=32,
                shuffle=False
            )
            # =========================
            
            # load model
            # =========================
            pretrained_model = pretrained_model_chosen(
                input_shape=(224, 224, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

            pretrained_model.trainable = False

            inputs = pretrained_model.input
            x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
            x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

            outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
            model = tf.keras.Model(inputs, outputs)
            # print(model.summary())
            # =========================


            
            ## compile and evaluate
            # =========================

            print("-------Training " + save_path + "_" + concat_type + "-------")
            model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

            ## early stop 
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
            model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30, verbose=0)
            ## no early stop
            # history = model.fit(train_images, validation_data=val_images, epochs=30)

            model.evaluate(test_images, verbose=0)
            # =========================

            pred = model.predict(test_images)

            ## 把pred分開存
            if save_path == "front":
                front_pred = pred
            else:
                side_pred = pred
            predicted_labels = np.argmax(pred, axis=1)
        # =========================
            
        ## print each fold results
        # =========================
        ## 做voting
        for i in range(len(vote_type_arr)):
            print("------- "+vote_type_arr[i]+" -------")
            if vote_type_arr[i] == "front":
                predicted_labels = np.argmax(front_pred, axis=1)
            if vote_type_arr[i] == "side":
                predicted_labels = np.argmax(side_pred, axis=1)
            if vote_type_arr[i] == "soft":
                soft_pred = np.argmax((front_pred + side_pred) / 2 , axis=1)
                predicted_labels = soft_pred
            if vote_type_arr[i] == "hard":
                hard_front_pred = np.argmax(front_pred, axis=1)  
                hard_side_pred = np.argmax(side_pred, axis=1)  
                hard_pred = mode([hard_front_pred, hard_side_pred], axis=0).mode[0]  
                predicted_labels = hard_pred
            # if vote_type_arr[i] == "product":
            #     product_pred = [[a * b for a, b in zip(sublist1, sublist2)] for sublist1, sublist2 in zip(front_pred, side_pred)]
            #     predicted_labels = product_pred

            acc = accuracy_score(test_images.labels, predicted_labels)
            f1 = f1_score(test_images.labels, predicted_labels, average='macro')
            # precision = precision_score(test_images.labels, predicted_labels, average='macro')
            # recall = recall_score(test_images.labels, predicted_labels, average='macro')

            print(f"Test Accuracy: {np.round(acc, 2)}")
            print(f"f1 score: {np.round(f1, 2)}")
            # print(f"precision: {np.round(precision, 2)}")
            # print(f"recall: {np.round(recall, 2)}")

            ## 存起來要做統計
            vote_type_arr_acc[i].append(np.round(acc, 2))
            vote_type_arr_f1[i].append(np.round(f1, 2))

            # print(vote_type_arr_acc)
            # print(vote_type_arr_f1)

        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    acc_mean = []
    acc_std = []
    f1_mean = []
    f1_std = []
    # =========================
    for i in range(len(vote_type_arr)):
        acc_mean.append(np.round(np.mean(vote_type_arr_acc[i]), 2))
        acc_std.append(np.round(np.std(vote_type_arr_acc[i]), 2))
        f1_mean.append(np.round(np.mean(vote_type_arr_f1[i]), 2))
        f1_std.append(np.round(np.std(vote_type_arr_f1[i]), 2))
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':vote_type_arr_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':vote_type_arr_f1}


path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"

concat_type = "voting"
side_pred = ""
front_pred = ""

# =========================





results = train_cross_validation(image_dir=path, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  

In [ ]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================


    # ## load chosen model
    # # =========================
    # if chosen_model == 'resnet50':
    #     preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # # elif chosen_model == 'resnet152':
    # #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    # #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    # elif chosen_model == 'vgg16':
    #     preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    # elif chosen_model == 'vgg19':
    #     preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    # elif chosen_model == 'mobilenet':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    # elif chosen_model == 'mobilenet_v2':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    # elif chosen_model == 'efficientnet':
    #     preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    # elif chosen_model == 'inception_v3':
    #     preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    # else:
    #     raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    # test_acc = []
    # test_f1 = []
    front_test_acc = []
    side_test_acc = []
    soft_test_acc = []
    hard_test_acc = []

    front_test_f1 = []
    side_test_f1 = []
    soft_test_f1 = []
    hard_test_f1 = []
    
    vote_type_arr_acc = [front_test_acc, side_test_acc, soft_test_acc, hard_test_acc]
    vote_type_arr_f1 = [front_test_f1, side_test_f1, soft_test_f1, hard_test_f1]
    vote_type_arr = ["front", "side", "soft", "hard"]


    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]


        ## train, validation image
        # =========================
        ## train front and side
        save_path_arr = ["front", "side"]
        for save_path in save_path_arr:
            ## 分資料
            # =========================
            if save_path == "side":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("front", "side")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("front", "side")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("front", "side")
                preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
                pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0

            if save_path == "front":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("side", "front")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("side", "front")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("side", "front")
                preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
                pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50

            print(len(k_fold_train))
            print(len(k_fold_val))


            # 確認
            print(k_fold_train.iloc[0]['Filepath'].split("\\")[-3])
            print(test_df.iloc[0]['Filepath'].split("\\")[-3])
            
            ## 讀資料
            # =========================
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
            train_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_train,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            val_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_val,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
            test_images = test_generator.flow_from_dataframe(
                dataframe=test_df,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=32,
                shuffle=False
            )
            # =========================
            
            # load model
            # =========================
            pretrained_model = pretrained_model_chosen(
                input_shape=(224, 224, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

            pretrained_model.trainable = False

            inputs = pretrained_model.input
            x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
            x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

            outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
            model = tf.keras.Model(inputs, outputs)
            # print(model.summary())
            # =========================


            
            ## compile and evaluate
            # =========================

            print("-------Training " + save_path + "_" + concat_type + "-------")
            model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

            ## early stop 
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
            model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30, verbose=0)
            ## no early stop
            # history = model.fit(train_images, validation_data=val_images, epochs=30)

            model.evaluate(test_images, verbose=0)
            # =========================

            pred = model.predict(test_images)

            ## 把pred分開存
            if save_path == "front":
                front_pred = pred
            else:
                side_pred = pred
            predicted_labels = np.argmax(pred, axis=1)
        # =========================
            
        ## print each fold results
        # =========================
        ## 做voting
        for i in range(len(vote_type_arr)):
            print("------- "+vote_type_arr[i]+" -------")
            if vote_type_arr[i] == "front":
                predicted_labels = np.argmax(front_pred, axis=1)
            if vote_type_arr[i] == "side":
                predicted_labels = np.argmax(side_pred, axis=1)
            if vote_type_arr[i] == "soft":
                soft_pred = np.argmax((front_pred + side_pred) / 2 , axis=1)
                predicted_labels = soft_pred
            if vote_type_arr[i] == "hard":
                hard_front_pred = np.argmax(front_pred, axis=1)  
                hard_side_pred = np.argmax(side_pred, axis=1)  
                hard_pred = mode([hard_front_pred, hard_side_pred], axis=0).mode[0]  
                predicted_labels = hard_pred
            
            acc = accuracy_score(test_images.labels, predicted_labels)
            f1 = f1_score(test_images.labels, predicted_labels, average='macro')
            # precision = precision_score(test_images.labels, predicted_labels, average='macro')
            # recall = recall_score(test_images.labels, predicted_labels, average='macro')

            print(f"Test Accuracy: {np.round(acc, 2)}")
            print(f"f1 score: {np.round(f1, 2)}")
            # print(f"precision: {np.round(precision, 2)}")
            # print(f"recall: {np.round(recall, 2)}")

            ## 存起來要做統計
            vote_type_arr_acc[i].append(np.round(acc, 2))
            vote_type_arr_f1[i].append(np.round(f1, 2))

            # print(vote_type_arr_acc)
            # print(vote_type_arr_f1)

        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    acc_mean = []
    acc_std = []
    f1_mean = []
    f1_std = []
    # =========================
    for i in range(len(vote_type_arr)):
        acc_mean.append(np.round(np.mean(vote_type_arr_acc[i]), 2))
        acc_std.append(np.round(np.std(vote_type_arr_acc[i]), 2))
        f1_mean.append(np.round(np.mean(vote_type_arr_f1[i]), 2))
        f1_std.append(np.round(np.std(vote_type_arr_f1[i]), 2))
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':vote_type_arr_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':vote_type_arr_f1}


path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"

concat_type = "voting"
side_pred = ""
front_pred = ""

# =========================





results = train_cross_validation(image_dir=path, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  

In [ ]:
# import
import numpy as np
import pandas as pd
import os.path
# import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
import matplotlib.pyplot as plt
import os
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, precision_score, recall_score

## label
# =========================
def class_2_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    else:
        label = "1"
    return label

def class_3_type(root):
    label = ""
    if "正常" in root:
        label = "0"
    elif "雙踝" in root:
        label = "1"
    elif "三踝" in root:
        label = "2"
    return label
# =========================


def load_path(path, class_count):
    dataset = []
    class_type = ''
    if class_count == 2:
        class_type = class_2_type
    elif class_count == 3:
        class_type = class_3_type  

    for root, dirs, files in os.walk(path):
        for file in files:
            label = class_type(root)
            # if label != "":
            dataset.append(
                            {   
                                'uuid': root.split("\\")[-1],
                                'label': label,
                                'image_path': os.path.join(root, file)
                            }
                        )

    return dataset

# def train_cross_validation(image_dir, n_splits=5, class_count=2, maru_part=None,  chosen_model='resnet50'):
def train_cross_validation(image_dir, n_splits=5, class_count=2):


    ## load data and  labels
    # =========================
    labels = []
    filepaths = []
    data = load_path(image_dir, class_count)
    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)
    # =========================


    # ## load chosen model
    # # =========================
    # if chosen_model == 'resnet50':
    #     preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50
    # # elif chosen_model == 'resnet152':
    # #     preprocessing_function_chosen = tf.keras.applications.resnet152.preprocess_input
    # #     pretrained_model_chosen = tf.keras.applications.resnet152.ResNet152
    # elif chosen_model == 'vgg16':
    #     preprocessing_function_chosen = tf.keras.applications.vgg16.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg16.VGG16
    # elif chosen_model == 'vgg19':
    #     preprocessing_function_chosen = tf.keras.applications.vgg19.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.vgg19.VGG19
    # elif chosen_model == 'mobilenet':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet.MobileNet
    # elif chosen_model == 'mobilenet_v2':
    #     preprocessing_function_chosen = tf.keras.applications.mobilenet_v2.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.mobilenet_v2.MobileNetV2
    # elif chosen_model == 'efficientnet':
    #     preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0
    # elif chosen_model == 'inception_v3':
    #     preprocessing_function_chosen = tf.keras.applications.inception_v3.preprocess_input
    #     pretrained_model_chosen = tf.keras.applications.inception_v3.InceptionV3
    # else:
    #     raise ValueError("Model name not recognized. Please choose a valid model.")




    # =========================

    ## to do: loss function
    # =========================
    # =========================


    train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=1, stratify=images['Label'])


    ## train model, k fold cross validation
    # ==================================================
    fold_no = 1
    # test_acc = []
    # test_f1 = []
    front_test_acc = []
    side_test_acc = []
    soft_test_acc = []
    hard_test_acc = []

    front_test_f1 = []
    side_test_f1 = []
    soft_test_f1 = []
    hard_test_f1 = []
    
    vote_type_arr_acc = [front_test_acc, side_test_acc, soft_test_acc, hard_test_acc]
    vote_type_arr_f1 = [front_test_f1, side_test_f1, soft_test_f1, hard_test_f1]
    vote_type_arr = ["front", "side", "soft", "hard"]


    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)
    for train_index, val_index in kfold.split(train_df, train_df['Label']):
        k_fold_train = train_df.iloc[train_index]
        k_fold_val = train_df.iloc[val_index]


        ## train, validation image
        # =========================
        ## train front and side
        save_path_arr = ["front", "side"]
        for save_path in save_path_arr:
            ## 分資料
            # =========================
            if save_path == "side":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("front", "side")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("front", "side")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("front", "side")
                preprocessing_function_chosen = tf.keras.applications.efficientnet.preprocess_input
                pretrained_model_chosen = tf.keras.applications.efficientnet.EfficientNetB0

            if save_path == "front":
                k_fold_train.loc[:, "Filepath"] = k_fold_train["Filepath"].str.replace("side", "front")
                k_fold_val.loc[:, "Filepath"] = k_fold_val["Filepath"].str.replace("side", "front")
                test_df.loc[:, "Filepath"] = test_df["Filepath"].str.replace("side", "front")
                preprocessing_function_chosen = tf.keras.applications.resnet50.preprocess_input
                pretrained_model_chosen = tf.keras.applications.resnet50.ResNet50

            print(len(k_fold_train))
            print(len(k_fold_val))


            # 確認
            print(k_fold_train.iloc[0]['Filepath'].split("\\")[-3])
            print(test_df.iloc[0]['Filepath'].split("\\")[-3])
            
            ## 讀資料
            # =========================
            train_generator = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=False,preprocessing_function=preprocessing_function_chosen)
            train_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_train,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            val_images = train_generator.flow_from_dataframe(
                dataframe=k_fold_val,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=64,
                shuffle=True,
                seed=42,
            )
            test_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessing_function_chosen)
            test_images = test_generator.flow_from_dataframe(
                dataframe=test_df,
                x_col='Filepath',
                y_col='Label',
                target_size=(224, 224),
                color_mode='rgb',
                class_mode='categorical',
                batch_size=32,
                shuffle=False
            )
            # =========================
            
            # load model
            # =========================
            pretrained_model = pretrained_model_chosen(
                input_shape=(224, 224, 3),
                include_top=False,
                weights='imagenet',
                pooling='avg')

            pretrained_model.trainable = False

            inputs = pretrained_model.input
            x = tf.keras.layers.Dense(128, activation='relu', name='dense_128')(pretrained_model.output)
            x = tf.keras.layers.Dense(50, activation='relu', name='dense_50')(x)

            outputs = tf.keras.layers.Dense(class_count, activation='softmax', name='output_layer')(x)
            model = tf.keras.Model(inputs, outputs)
            # print(model.summary())
            # =========================


            
            ## compile and evaluate
            # =========================

            print("-------Training " + save_path + "_" + concat_type + "-------")
            model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

            ## early stop 
            # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
            # model.fit(train_images, validation_data=val_images, callbacks=[early_stopping], epochs=30)
            ## no early stop
            model.fit(train_images, validation_data=val_images, epochs=30, verbose=0)

            model.evaluate(test_images, verbose=0)
            # =========================

            pred = model.predict(test_images)

            ## 把pred分開存
            if save_path == "front":
                front_pred = pred
            else:
                side_pred = pred
            predicted_labels = np.argmax(pred, axis=1)
        # =========================
            
        ## print each fold results
        # =========================
        ## 做voting
        for i in range(len(vote_type_arr)):
            print("------- "+vote_type_arr[i]+" -------")
            if vote_type_arr[i] == "front":
                predicted_labels = np.argmax(front_pred, axis=1)
            if vote_type_arr[i] == "side":
                predicted_labels = np.argmax(side_pred, axis=1)
            if vote_type_arr[i] == "soft":
                soft_pred = np.argmax((front_pred + side_pred) / 2 , axis=1)
                predicted_labels = soft_pred
            if vote_type_arr[i] == "hard":
                hard_front_pred = np.argmax(front_pred, axis=1)  
                hard_side_pred = np.argmax(side_pred, axis=1)  
                hard_pred = mode([hard_front_pred, hard_side_pred], axis=0).mode[0]  
                predicted_labels = hard_pred
            
            acc = accuracy_score(test_images.labels, predicted_labels)
            f1 = f1_score(test_images.labels, predicted_labels, average='macro')
            # precision = precision_score(test_images.labels, predicted_labels, average='macro')
            # recall = recall_score(test_images.labels, predicted_labels, average='macro')

            print(f"Test Accuracy: {np.round(acc, 2)}")
            print(f"f1 score: {np.round(f1, 2)}")
            # print(f"precision: {np.round(precision, 2)}")
            # print(f"recall: {np.round(recall, 2)}")

            ## 存起來要做統計
            vote_type_arr_acc[i].append(np.round(acc, 2))
            vote_type_arr_f1[i].append(np.round(f1, 2))

            # print(vote_type_arr_acc)
            # print(vote_type_arr_f1)

        fold_no += 1
        # =========================
    # ==================================================
        
    ## print  mean results
    acc_mean = []
    acc_std = []
    f1_mean = []
    f1_std = []
    # =========================
    for i in range(len(vote_type_arr)):
        acc_mean.append(np.round(np.mean(vote_type_arr_acc[i]), 2))
        acc_std.append(np.round(np.std(vote_type_arr_acc[i]), 2))
        f1_mean.append(np.round(np.mean(vote_type_arr_f1[i]), 2))
        f1_std.append(np.round(np.std(vote_type_arr_f1[i]), 2))
    # print(f"acc mean = {acc_mean}, std = {acc_std}")
    # print(test_acc)
    # print(f"f1  mean = {f1_mean}, std = {f1_std}")
    # print(test_f1)
    # =========================

    return {'acc_mean':acc_mean, 'acc_std':acc_std, 'test_acc':vote_type_arr_acc, 'f1_mean':f1_mean, 'f1_std':f1_std, 'test_f1':vote_type_arr_f1}


path = "E:\\data_bone\\9-a+b_swift_cut_正確_V2\\front"

concat_type = "voting"
side_pred = ""
front_pred = ""

# =========================





results = train_cross_validation(image_dir=path, class_count=3)


print(f"acc mean = {results['acc_mean']}, std = {results['acc_std']}, test_acc = {results['test_acc']}")
print(f"f1  mean = {results['f1_mean']}, std = {results['f1_std']}, test_f1 = {results['test_f1']}")
  